# Segment geospatial - Generating object masks from input prompts with HQ-SAM

Segment geospatial [input prompts HQ Doku](https://samgeo.gishub.org/examples/input_prompts_hq/)

In [1]:
import os
import cv2
import numpy as np
import torch
from matplotlib import pyplot as plt
import leafmap
from samgeo.hq_sam import SamGeo, tms_to_geotiff

Create an interactive map

In [2]:
m = leafmap.Map(height="700px", width="1000px")
m.add_basemap("Satellite")
# m.add_layer_manager()

In [3]:
image_name = '32691_5334'
image = f'../data/{image_name}.tif'

Display the image on the map.

In [4]:
m.layers[-1].visible = False
m.add_raster(image, layer_name="Image")
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

## Initialize SAM class

Specify the file path to the model checkpoint. If it is not specified, the model will to downloaded to the working directory.

Set `automatic=False` to disable the `SamAutomaticMaskGenerator` and enable the `SamPredictor`.

In [5]:
SAM_CHECKPOINT = "../segment-anything/checkpoints/sam_hq_vit_l.pth"
MODEL_TYPE = "vit_l"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

sam = SamGeo(
    model_type=MODEL_TYPE,
    checkpoint=SAM_CHECKPOINT,
    device=DEVICE,
    automatic=False
)

<All keys matched successfully>


Specify the image to segment. 

In [6]:
sam.set_image(image)

## Image segmentation with input points

A single point can be used to segment an object. The point can be specified as a tuple of (x, y), such as (col, row) or (lon, lat). The points can also be specified as a file path to a vector dataset. For non (col, row) input points, specify the `point_crs` parameter, which will automatically transform the points to the image column and row coordinates.

Try a single point input:

In [7]:
# point_coords = [(100, 300)]
# sam.predict(point_coords, point_labels=1, output="mask1.tif")
# m.add_raster("mask1.tif", layer_name="Mask1", nodata=0, cmap="Blues", opacity=1)
# m

Try multiple points input:

## Interactive segmentation

Display the interactive map and use the marker tool to draw points on the map. Then click on the `Segment` button to segment the objects. The results will be added to the map automatically. Click on the `Reset` button to clear the points and the results.

In [15]:
m = sam.show_map(height="700px", width="1000px")
m

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…